# Raw data to cleaned data (renaming columns, data type conversions, etc.)

In [1]:
from pyspark.sql import SparkSession

# Create a spark session (which will run spark jobs)
spark = (
    SparkSession.builder.appName("MAST30034 Project 1")
    .config("spark.sql.repl.eagerEval.enabled", True) 
    .config("spark.sql.parquet.cacheMetadata", "true")
    .config("spark.sql.session.timeZone", "Etc/UTC") # fix timestamps loaded by spark
    .getOrCreate()
)

your 131072x1 screen size is bogus. expect trouble
23/07/30 15:30:36 WARN Utils: Your hostname, DESKTOP-SATV84A resolves to a loopback address: 127.0.1.1; using 172.26.254.29 instead (on interface eth0)
23/07/30 15:30:36 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/07/30 15:30:38 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Yellow Taxis

In [2]:
# make directories for cleaned data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/yellow'):
    os.makedirs('../data/cleaned/tlc/yellow')
    
if not os.path.exists('../data/cleaned/tlc/yellow/2022'):
    os.makedirs('../data/cleaned/tlc/yellow/2022')
    
if not os.path.exists('../data/cleaned/tlc/yellow/2023'):
    os.makedirs('../data/cleaned/tlc/yellow/2023')

### Check data types

In [15]:
yellow_raw_dir = '../data/raw/tlc/yellow/'

sdf_yellow_2022_01 = spark.read.parquet(f"{yellow_raw_dir}2022/2022-01.parquet")

sdf_yellow_2022_all = spark.read.parquet(f"{yellow_raw_dir}2022")

sdf_yellow_2023_01 = spark.read.parquet(f"{yellow_raw_dir}2023/2023-01.parquet")

sdf_yellow_2023_02 = spark.read.parquet(f"{yellow_raw_dir}2023/2023-02.parquet")

#sdf_yellow_2022_all = spark.read.parquet('../data/raw/yellow/2022')



In [16]:
sdf_yellow_2022_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [42]:
sdf_yellow_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('RatecodeID', DoubleType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [ ]:
sdf_yellow_2023_01.schema

In [ ]:
sdf_yellow_2023_02.schema

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [2]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# RatecodeID and passenger_count also need to be converted to int

sdf_yellow_2023_02 = sdf_yellow_2023_02.withColumn(
    'ratecodeid', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_yellow_2023_02 = sdf_yellow_2023_02.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_yellow_2023_02.schema



NameError: name 'sdf_yellow_2023_02' is not defined

In [18]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_yellow_2023_02.columns]
sdf_yellow_2023_02 = sdf_yellow_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_yellow_2023_02.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('tpep_pickup_datetime', TimestampNTZType(), True), StructField('tpep_dropoff_datetime', TimestampNTZType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('ratecodeid', IntegerType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('payment_type', LongType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True)])

In [22]:
# convert all raw datasets to required schema

yellow_raw_dir = '../data/raw/tlc/yellow/'
    
yellow_clean_dir = '../data/cleaned/tlc/yellow/'

# 2022
for month in range(1,13):

    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{yellow_raw_dir}2022/2022-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_clean_dir}2022/2022-{month}.parquet")

# 2023
for month in range(1,5):
    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{yellow_raw_dir}2023/2023-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{yellow_clean_dir}2023/2023-{month}.parquet")


In [8]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.parquet('../data/cleaned/tlc/yellow/2022/2022-01.parquet')

sdf1.show(1, vertical=True, truncate=100)

#sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 tpep_pickup_datetime  | 2022-01-01 00:35:40 
 tpep_dropoff_datetime | 2022-01-01 00:53:29 
 passenger_count       | 2                   
 trip_distance         | 3.8                 
 ratecodeid            | 1                   
 store_and_fwd_flag    | N                   
 pulocationid          | 142                 
 dolocationid          | 236                 
 payment_type          | 1                   
 fare_amount           | 14.5                
 extra                 | 3.0                 
 mta_tax               | 0.5                 
 tip_amount            | 3.65                
 tolls_amount          | 0.0                 
 improvement_surcharge | 0.3                 
 total_amount          | 21.95               
 congestion_surcharge  | 2.5                 
 airport_fee           | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [ ]:
# double check schema of 2023 cleaned data

sdf = spark.read.parquet('../data/cleaned/tlc/yellow/2023/*')

sdf.show(1, vertical=True, truncate=100)

## Green Taxis

In [9]:
# make directories for cleaned data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/green'):
    os.makedirs('../data/cleaned/tlc/green')
    
if not os.path.exists('../data/cleaned/tlc/green/2022'):
    os.makedirs('../data/cleaned/tlc/green/2022')
    
if not os.path.exists('../data/cleaned/tlc/green/2023'):
    os.makedirs('../data/cleaned/tlc/green/2023')

### Check data types

In [10]:
green_raw_dir = '../data/raw/tlc/green/'

sdf_green_2022_01 = spark.read.parquet(f"{green_raw_dir}2022/2022-01.parquet")

sdf_green_2022_all = spark.read.parquet(f"{green_raw_dir}2022")

sdf_green_2023_01 = spark.read.parquet(f"{green_raw_dir}2023/2023-01.parquet")

sdf_green_2023_02 = spark.read.parquet(f"{green_raw_dir}2023/2023-02.parquet")

#sdf_green_2022_all = spark.read.parquet('../data/raw/green/2022')



In [46]:
sdf_green_2022_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [47]:
sdf_green_2022_all.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [48]:
sdf_green_2023_01.schema

StructType([StructField('VendorID', LongType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', DoubleType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('passenger_count', DoubleType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', IntegerType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', DoubleType(), True), StructField('trip_type', DoubleType(), True), StructField('congestion_surcharge', Doubl

In [49]:
sdf_green_2023_02.schema

StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('RatecodeID', LongType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', LongType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge', Doubl

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [11]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# RatecodeID and passenger_count also need to be converted to int

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'ratecodeid', 
    F.col('RatecodeID').cast('Integer') 
)

sdf_green_2023_02 = sdf_green_2023_02.withColumn(
    'passenger_count',
    F.col('passenger_count').cast('Integer')
)


sdf_green_2023_02.schema



StructType([StructField('VendorID', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [12]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_green_2023_02.columns]
sdf_green_2023_02 = sdf_green_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_green_2023_02.schema
sdf_schema

StructType([StructField('vendorid', IntegerType(), True), StructField('lpep_pickup_datetime', TimestampNTZType(), True), StructField('lpep_dropoff_datetime', TimestampNTZType(), True), StructField('store_and_fwd_flag', StringType(), True), StructField('ratecodeid', IntegerType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('passenger_count', IntegerType(), True), StructField('trip_distance', DoubleType(), True), StructField('fare_amount', DoubleType(), True), StructField('extra', DoubleType(), True), StructField('mta_tax', DoubleType(), True), StructField('tip_amount', DoubleType(), True), StructField('tolls_amount', DoubleType(), True), StructField('ehail_fee', DoubleType(), True), StructField('improvement_surcharge', DoubleType(), True), StructField('total_amount', DoubleType(), True), StructField('payment_type', LongType(), True), StructField('trip_type', LongType(), True), StructField('congestion_surcharge',

In [13]:
# convert all raw datasets to required schema

green_raw_dir = '../data/raw/tlc/green/'
    
green_clean_dir = '../data/cleaned/tlc/green/'

# 2022
for month in range(1,13):

    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{green_raw_dir}2022/2022-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{green_clean_dir}2022/2022-{month}.parquet")

# 2023
for month in range(1,5):
    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{green_raw_dir}2023/2023-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{green_clean_dir}2023/2023-{month}.parquet")


In [15]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.parquet('../data/cleaned/tlc/green/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0------------------------------------
 vendorid              | 1                   
 lpep_pickup_datetime  | 2022-03-01 00:24:14 
 lpep_dropoff_datetime | 2022-03-01 00:34:04 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 74                  
 dolocationid          | 151                 
 passenger_count       | 1                   
 trip_distance         | 2.3                 
 fare_amount           | 9.5                 
 extra                 | 0.5                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 0.3                 
 total_amount          | 10.8                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [16]:
# double check schema of 2023 cleaned data

sdf = spark.read.parquet('../data/cleaned/tlc/green/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0------------------------------------
 vendorid              | 2                   
 lpep_pickup_datetime  | 2023-03-01 00:25:10 
 lpep_dropoff_datetime | 2023-03-01 00:35:47 
 store_and_fwd_flag    | N                   
 ratecodeid            | 1                   
 pulocationid          | 82                  
 dolocationid          | 196                 
 passenger_count       | 1                   
 trip_distance         | 2.36                
 fare_amount           | 13.5                
 extra                 | 1.0                 
 mta_tax               | 0.5                 
 tip_amount            | 0.0                 
 tolls_amount          | 0.0                 
 ehail_fee             | null                
 improvement_surcharge | 1.0                 
 total_amount          | 16.0                
 payment_type          | 2                   
 trip_type             | 1                   
 congestion_surcharge  | 0.0                 
only showing top 1 row



## High Volume For-Hire Vehicles

In [17]:
# make directories for cleaned data

import os
from pyspark.sql import functions as F

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv'):
    os.makedirs('../data/cleaned/tlc/fhvhv')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv/2022'):
    os.makedirs('../data/cleaned/tlc/fhvhv/2022')
    
if not os.path.exists('../data/cleaned/tlc/fhvhv/2023'):
    os.makedirs('../data/cleaned/tlc/fhvhv/2023')

### Check data types

In [18]:
fhvhv_raw_dir = '../data/raw/tlc/fhvhv/'

sdf_fhvhv_2022_01 = spark.read.parquet(f"{fhvhv_raw_dir}2022/2022-01.parquet")

sdf_fhvhv_2022_all = spark.read.parquet(f"{fhvhv_raw_dir}2022")

sdf_fhvhv_2023_01 = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-01.parquet")

sdf_fhvhv_2023_02 = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-02.parquet")

#sdf_fhvhv_2022_all = spark.read.parquet(f"{fhvhv_raw_dir}2022")



In [58]:
sdf_fhvhv_2022_01.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [59]:
sdf_fhvhv_2022_all.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [60]:
sdf_fhvhv_2023_01.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', LongType(), True), StructField('DOLocationID', LongType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('shared_re

In [61]:
sdf_fhvhv_2023_02.schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', LongType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('sha

### Schemas appear to be incorrect for all of 2022, so we will adjust all datasets to line up with the schema of Feb 2023 data and ensure consistent lowercase casing

In [19]:
'''
# renaming
sdf.withColumnRenamed(
    'column_from',
    'column_to'
)

# example 1 for converting data types
sdf.withColumn(
    'column_to',
    F.col('column_from').cast('data type')
)
'''

# trip_time needs to be converted to integer

sdf_fhvhv_2023_02 = sdf_fhvhv_2023_02.withColumn(
    'trip_time', 
    F.col('trip_time').cast('Integer') 
)

sdf_fhvhv_2023_02.schema



StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('PULocationID', IntegerType(), True), StructField('DOLocationID', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', IntegerType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('

In [20]:
# now, we want to ensure everything has consistent casing to make our lives easier
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in sdf_fhvhv_2023_02.columns]
sdf_fhvhv_2023_02 = sdf_fhvhv_2023_02.select(*consistent_col_casing)

# this will be used in the cell below when reading in
sdf_schema = sdf_fhvhv_2023_02.schema
sdf_schema

StructType([StructField('hvfhs_license_num', StringType(), True), StructField('dispatching_base_num', StringType(), True), StructField('originating_base_num', StringType(), True), StructField('request_datetime', TimestampNTZType(), True), StructField('on_scene_datetime', TimestampNTZType(), True), StructField('pickup_datetime', TimestampNTZType(), True), StructField('dropoff_datetime', TimestampNTZType(), True), StructField('pulocationid', IntegerType(), True), StructField('dolocationid', IntegerType(), True), StructField('trip_miles', DoubleType(), True), StructField('trip_time', IntegerType(), True), StructField('base_passenger_fare', DoubleType(), True), StructField('tolls', DoubleType(), True), StructField('bcf', DoubleType(), True), StructField('sales_tax', DoubleType(), True), StructField('congestion_surcharge', DoubleType(), True), StructField('airport_fee', DoubleType(), True), StructField('tips', DoubleType(), True), StructField('driver_pay', DoubleType(), True), StructField('

In [21]:
# convert all raw datasets to required schema

fhvhv_raw_dir = '../data/raw/tlc/fhvhv/'
    
fhvhv_clean_dir = '../data/cleaned/tlc/fhvhv/'

# 2022
for month in range(1,13):

    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{fhvhv_raw_dir}2022/2022-{month}.parquet")

    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{fhvhv_clean_dir}2022/2022-{month}.parquet")

# 2023
for month in range(1,5):
    month = str(month).zfill(2) 

    sdf_malformed = spark.read.parquet(f"{fhvhv_raw_dir}2023/2023-{month}.parquet")
 
    # select all columns from the existing malformed dataframe and cast it to the required schema and change casing
    sdf_malformed = sdf_malformed \
        .select([F.col(c).cast(sdf_schema[i].dataType) for i, c in enumerate(sdf_malformed.columns)]) \
        .select(*consistent_col_casing)
        
        
    sdf_malformed \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet(f"{fhvhv_clean_dir}2023/2023-{month}.parquet")


In [22]:
# double check schema of 2022 cleaned data

sdf1 = spark.read.parquet('../data/cleaned/tlc/fhvhv/2022/*')

sdf1.show(1, vertical=True, truncate=100)

''' checking raw vs cleaned dataset shape and raw dataset shape of jan 2022
sdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')

sdf2.show(1, vertical=True, truncate=100)

print(sdf1.count())

print(sdf2.count())

sdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')

sdf.show(1, vertical=True, truncate=100)

print(sdf.count())
'''

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2022-01-01 00:05:31 
 on_scene_datetime    | 2022-01-01 00:05:40 
 pickup_datetime      | 2022-01-01 00:07:24 
 dropoff_datetime     | 2022-01-01 00:18:28 
 pulocationid         | 170                 
 dolocationid         | 161                 
 trip_miles           | 1.18                
 trip_time            | 664                 
 base_passenger_fare  | 24.9                
 tolls                | 0.0                 
 bcf                  | 0.75                
 sales_tax            | 2.21                
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 0.0                 
 driver_pay           | 23.03               
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

" checking raw vs cleaned dataset shape and raw dataset shape of jan 2022\nsdf2 = spark.read.parquet('../data/raw/tlc/yellow/2022')\n\nsdf2.show(1, vertical=True, truncate=100)\n\nprint(sdf1.count())\n\nprint(sdf2.count())\n\nsdf = spark.read.parquet('../data/raw/tlc/yellow/2022/2022-01.parquet')\n\nsdf.show(1, vertical=True, truncate=100)\n\nprint(sdf.count())\n"

In [24]:
# double check schema of 2023 cleaned data

sdf = spark.read.parquet('../data/cleaned/tlc/fhvhv/2023/*')

sdf.show(1, vertical=True, truncate=100)

-RECORD 0-----------------------------------
 hvfhs_license_num    | HV0003              
 dispatching_base_num | B03404              
 originating_base_num | B03404              
 request_datetime     | 2023-01-01 00:18:06 
 on_scene_datetime    | 2023-01-01 00:19:24 
 pickup_datetime      | 2023-01-01 00:19:38 
 dropoff_datetime     | 2023-01-01 00:48:07 
 pulocationid         | 48                  
 dolocationid         | 68                  
 trip_miles           | 0.94                
 trip_time            | 1709                
 base_passenger_fare  | 25.95               
 tolls                | 0.0                 
 bcf                  | 0.78                
 sales_tax            | 2.3                 
 congestion_surcharge | 2.75                
 airport_fee          | 0.0                 
 tips                 | 5.22                
 driver_pay           | 27.83               
 shared_request_flag  | N                   
 shared_match_flag    | N                   
 access_a_

## Taxi Zones

In [75]:
# make directories for cleaned data

import os

if not os.path.exists('../data/cleaned/'):
    os.makedirs('../data/cleaned/')
    
if not os.path.exists('../data/cleaned/tlc'):
    os.makedirs('../data/cleaned/tlc')
    
if not os.path.exists('../data/cleaned/tlc/taxi_zones'):
    os.makedirs('../data/cleaned/tlc/taxi_zones')

In [5]:
taxi_zone_lookup = spark.read.option("header",True) \
                   .csv("../data/raw/tlc/taxi_zones/taxi+_zone_lookup.csv")

In [10]:
print(taxi_zone_lookup.limit(5))

print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|LocationID|      Borough|                Zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('LocationID', StringType(), True), StructField('Borough', StringType(), True), StructField('Zone', StringType(), True), StructField('service_zone', StringType(), True)])


### Just need to convert the column names to lowercase for this data

In [12]:
from pyspark.sql import functions as F

# converting to lower case
consistent_col_casing = [F.col(col_name).alias(col_name.lower()) for col_name in taxi_zone_lookup.columns]
taxi_zone_lookup = taxi_zone_lookup.select(*consistent_col_casing)

print(taxi_zone_lookup.limit(5))
print(taxi_zone_lookup.schema)

+----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
+----------+-------------+--------------------+------------+

StructType([StructField('locationid', StringType(), True), StructField('borough', StringType(), True), StructField('zone', StringType(), True), StructField('service_zone', StringType(), True)])


In [13]:
# save cleaned data as parquet
taxi_zone_lookup \
    .coalesce(1) \
    .write \
    .mode('overwrite') \
    .parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

In [14]:
# check cleaned data
taxi_zone_lookup = spark.read.parquet('../data/cleaned/tlc/taxi_zones/taxi_zone_lookup.parquet')

taxi_zone_lookup.printSchema


<bound method DataFrame.printSchema of +----------+-------------+--------------------+------------+
|locationid|      borough|                zone|service_zone|
+----------+-------------+--------------------+------------+
|         1|          EWR|      Newark Airport|         EWR|
|         2|       Queens|         Jamaica Bay|   Boro Zone|
|         3|        Bronx|Allerton/Pelham G...|   Boro Zone|
|         4|    Manhattan|       Alphabet City| Yellow Zone|
|         5|Staten Island|       Arden Heights|   Boro Zone|
|         6|Staten Island|Arrochar/Fort Wad...|   Boro Zone|
|         7|       Queens|             Astoria|   Boro Zone|
|         8|       Queens|        Astoria Park|   Boro Zone|
|         9|       Queens|          Auburndale|   Boro Zone|
|        10|       Queens|        Baisley Park|   Boro Zone|
|        11|     Brooklyn|          Bath Beach|   Boro Zone|
|        12|    Manhattan|        Battery Park| Yellow Zone|
|        13|    Manhattan|   Battery Park City

In [27]:
# move taxi_zones.zip to cleaned data
import shutil

src = '../data/raw/tlc/taxi_zones/taxi_zones.zip'

dst = '../data/cleaned/tlc/taxi_zones/taxi_zones.zip'

shutil.copyfile(src, dst)

import zipfile

# unzip shapefile
with zipfile.ZipFile(dst, 'r') as zip_ref:
    zip_ref.extractall('../data/cleaned/tlc/taxi_zones')